In [ ]:
# !pip install sentence_transformers

In [ ]:
# concise_paragraph_ee = ["Vikrant is a mid-aged male sales executive who is facing a moderate amount of emotional exhaustion. While his job requires a lot of effort and direct contact with people, his level of emotional exhaustion is still quite manageable. He feels emotionally drained a few times per month, and frustrated with his work a few times per year. He also feels like he works too hard and is stressed by working with people once a week. Despite the fact that he is facing emotional exhaustion, he is still able to manage it with a few strategies. He should focus on taking regular breaks from work and engaging in activities that help him relax and unwind. Additionally, he should also try to find ways to meaningfully connect with people in a way that doesn't feel draining. By combining rest, relaxation, and meaningful social connections, Vikrant can better manage his emotional exhaustion."]

In [ ]:
# concise_paragraph_dp = ["Vikrant is a mid-age male Sales Executive who, on the whole, exhibits a healthy relationship with his co-workers and team. He has answered all questions pertaining to depersonalization in a manner that indicates he is fairly connected with his team and colleagues and has a good level of empathy towards them. He has only been feeling tired a few times a month, and has had the impression that his team has made him responsible for their issues at a few times per month. His patience has only been at the end of its rope a few times per year, and he has not become more uncaring, insensitive, or afraid that his job is making him uncaring. To ensure Vikrant continues to have a healthy work relationship, he should make sure that he takes regular breaks during the day, takes the time to connect with his team/colleagues on a personal level, and makes sure to prioritize his own mental health and well-being."]

In [ ]:
# concise_paragraph_pa = ["Vikrant is a mid-aged male Sales Executive who is highly successful in his role. He is able to deal with the emotional and practical issues that come with the job in a calm and effective manner. He has strong communication skills and is able to easily understand the feelings of his team and colleagues. He often looks after their problems in an efficient and diplomatic manner. He also takes time to create a relaxed atmosphere with his team and colleagues. Once a week, he takes time to accomplish worthwhile tasks, feel full of energy, and feel refreshed when close to his team/colleagues. He also makes sure to take a few times a month to reflect on how his work positively influences people. His strategy to success includes communication, creating a relaxed atmosphere, taking the time to help his team and colleagues, and reflecting on how his work is contributing positively in the world."]

In [ ]:
concise_paragraph_ee = ["Akash is a young male Customer Representative who is feeling emotionally drained by his work due to the constant demands of his job. He feels frustrated by his work and believes that he works too hard. He also finds it stressful to be in direct contact with customers on a daily basis. With this in mind, it is important to create an emotional exhaustion strategy to address Akash's challenges in order to improve his overall performance and wellbeing. Strategies should focus on creating a healthy work-life balance, such as taking regular breaks, exercising and engaging in activities outside of work. Additionally, Akash should talk to a professional about his feelings so that he can develop coping strategies to manage his emotional exhaustion. Finally, mentors and colleagues should provide support and encouragement to help him stay motivated and energized."]

concise_paragraph_dp = ["Akash is a young age male Customer Representative who is displaying some signs of depersonalization. He feels he deals with his team/colleagues impersonally, as if they are objects every day. He also feels that his team/colleagues make him responsible for some of their problems every day. However, he also reports feeling tired when he gets up in the morning and has to face another day at work never, being at the end of his patience a few times per month, really not caring about what happens to some of his team/colleagues a few times per year, and having become more insensitive to people in the workplace a few times per year. Given his age and profession, Akash may be exhibiting signs of depersonalization related to work-related stress. Therefore, it is important to create a safe and supportive environment in the workplace where Akash and his colleagues can openly communicate and build relationships. He should also be encouraged to take regular breaks, practice mindfulness, and seek additional help if necessary. Through these strategies, Akash can learn to better manage his stress, foster meaningful relationships in the workplace, and ultimately prevent depersonalization."]

concise_paragraph_pa = ["Akash is a young, male customer representative who is eager to put his skills and knowledge to work for the benefit of others. He is able to accomplish many worthwhile things in his job, although he is only able to do so a few times per month. He looks after his team's problems very effectively, a few times per week, and is able to understand their feelings a few times per year. Akash is also able to handle emotional problems very calmly and create a relaxed atmosphere with his team, although he doesn't feel refreshed when he has been close to them. In order to continue to make a positive influence on people, he will need to focus on building his communication skills and developing relationships with his colleagues. Additionally, he should strive to find ways to motivate his team and use his energy to create a productive and inspiring work space."]


In [ ]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re

from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import unicodedata
import nltk
import re

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# df = pd.read_csv('/content/combined_data.csv')

df = pd.read_csv('/content/yt_videos_meta.csv')

In [ ]:
import pandas as pd
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Preprocessing function
# def preprocessing(text):
#     stop = nltk.corpus.stopwords.words('english')
#     lem = nltk.stem.WordNetLemmatizer()
#     text = text.lower()
#     words = [lem.lemmatize(w) for w in text.split() if w not in stop]
#     return words

def preprocess_text(text):

    stop = stopwords.words('english')
    lem = WordNetLemmatizer()
    # Normalize characters to remove diacritical marks
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())
    # Remove punctuation and split into words
    words = re.sub(r'[^\w\s]', '', text).split()
    # Lemmatize and remove stopwords
    return [lem.lemmatize(w) for w in words if w not in stop]

def preprocessing(text):
    stop = nltk.corpus.stopwords.words('english')                                    #stopwords
    lem = WordNetLemmatizer()                                                        #initializing lemmatizer
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore')            #ignoring unicode data
    .decode('utf-8', 'ignore')
    .lower())                                                                        #lower-casing
    words = re.sub(r'[^\w\s]', '', text).split()
    return [lem.lemmatize(w) for w in words if w not in stop]

def apply_preprocessing(df, text_column):

    df['preprocessed_text'] = df[text_column].apply(preprocessing)
    df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: ' '.join(x))
    return df

def concatenate_text_columns(df, column_names, new_column_name):

    df[new_column_name] = df[column_names].apply(lambda x: ' '.join(x), axis=1)
    return df

df["Video_Description"] = df["Video_Description"].apply(str) #video_desc
df["Video_Title"] = df["Video_Title"].apply(str) #video_title
text_column_names=["Video_Description", "Video_Title"]
# Concatenate specified text columns into a new column 'Full_Des'
df = concatenate_text_columns(df, text_column_names, "Full_Des")

# Convert the video length column to minutes
df['video_length_min'] = pd.to_timedelta(df['Video_Length']).dt.total_seconds() / 60. #video_length

# Filter the dataframe based on video length criteria
filtered_df = df[(df['video_length_min'] > 2) & (df['video_length_min'] < 25)]

# Sort the filtered dataframe by video length
df = filtered_df.sort_values('video_length_min')


# Apply preprocessing to 'Full_Des'
df = apply_preprocessing(df, "Full_Des")

# Filter out empty lists in the 'text' column
df = df[df['Full_Des'].map(len) > 0]

# df_key = df[df['Search_Term'].isin(filtered_keywords)]

# df_EE = df[df['domain'] == 'Emotional exhaustion']
# df_DP = df[df['domain'] == 'Depersonalization']
# df_PA = df[df['domain'] == 'Personal achievement']


In [ ]:
# Initialize the BERT model for sentence embeddings
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# # Create sentence pairs for encoding
sentence_pairs = [(text, concise_paragraph_ee) for text in df['Full_Des']]

# Encode the sentence pairs
embeddings = model.encode(sentence_pairs)

# print (model)

In [ ]:
# Extract the embedding for the burnout_summary
burnout_summary_embedding = model.encode([(concise_paragraph_ee, concise_paragraph_ee)])[0]

# Reshape the 'burnout_summary_embedding' to match the shape of 'embeddings'
burnout_summary_embedding = burnout_summary_embedding.reshape(1, -1)

# Calculate the cosine similarity between 'text' embeddings and 'concise_paragraph_embedding'
cosine_similarities = cosine_similarity(embeddings, burnout_summary_embedding)

#new df
df_ee = df.copy()

# Add the cosine similarities to the dataframe
df_ee['cosine_similarity'] = cosine_similarities.flatten()

# Sort the dataframe by cosine similarity in descending order
df_ee_sorted = df_ee.sort_values('cosine_similarity', ascending=False)
# df_ee_sorted['recomm_domain'] = 'EE'
df_ee_sorted = df_ee_sorted[df_ee_sorted['Total_Views'] > 1000]

In [ ]:
df_ee_sorted.head(5)

,Search_Term,Video_URL,Video_Title,Channel_Name,Total_Subscribers,Total_Views,Total_Likes,Total_Dislikes,Total_Comments,Video_Description,...,Video_Tags,Age_Restrictions,License,Captions_Available,search_term_video_url,domain,Full_Des,video_length_min,preprocessed_text,cosine_similarity
72,Building emotional resilience,https://www.youtube.com/watch?v=Lsd5oDUpDjc,Emotional Resiliency & Mental Toughness,SEALFIT,0,592942,9839,0,195,Website: https://www.sealfit.com\nFacebook: ht...,...,"['Masterty Program', 'SealFit Youtube Vignette...",NaN,youtube,NaN,Building emotional resilience_https://www.yout...,Emotional exhaustion,Website: https://www.sealfit.com\nFacebook: ht...,4.400000,website httpswwwsealfitcom facebook httpswwwfa...,0.886579
290,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.883736
55,Coping with work pressure,https://www.youtube.com/watch?v=XxhuvNV-ync,Lecture 6.1: Stress and Coping,PSYC 4183 Health Psychology,unknown,34511,332,0,12,Table of Contents:\n\n01:51 - Emotion-focused ...,...,[],NaN,youtube,NaN,Coping with work pressure_https://www.youtube....,Emotional exhaustion,Table of Contents:\n\n01:51 - Emotion-focused ...,12.950000,table content 0151 emotionfocused coping 0309 ...,0.873610
294,Improving communication skills,https://www.youtube.com/watch?v=X3MG4fEJiF0,5 Simple Tips To Improve Your Communication Sk...,Ranveer Allahbadia,unknown,3151594,170268,0,3157,हमारी Personality Development Playlist देखो : ...,...,"['hindi', 'communication skills in hindi', 'be...",NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,हमारी Personality Development Playlist देखो : ...,8.716667,personality development playlist httpswwwyoutu...,0.865808
346,Work-life balance strategies,https://www.youtube.com/watch?v=hPz34pKUAhk,LOWEST Stress and BEST Work Life Balance. Cybe...,"Josh Madakor - Tech, Education, Career",unknown,52242,1805,0,251,🔒Hands-On Cybersecurity / SOC Analyst Training...,...,"['cyber security', 'cybersecurity', 'cyber sec...",NaN,youtube,NaN,Work-life balance strategies_https://www.youtu...,Personal achievement,🔒Hands-On Cybersecurity / SOC Analyst Training...,6.183333,handson cybersecurity soc analyst training rea...,0.862628


In [ ]:
# Initialize the BERT model for sentence embeddings
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# Create sentence pairs for encoding
sentence_pairs_dp = [(text, concise_paragraph_dp) for text in df['Full_Des']]

# Encode the sentence pairs
embeddings_dp = model.encode(sentence_pairs_dp)

In [ ]:
# Extract the embedding for the burnout_summary
burnout_summary_embedding_dp = model.encode([(concise_paragraph_dp, concise_paragraph_dp)])[0]

# Reshape the 'burnout_summary_embedding' to match the shape of 'embeddings'
burnout_summary_embedding_dp = burnout_summary_embedding_dp.reshape(1, -1)

# Calculate the cosine similarity between 'text' embeddings and 'concise_paragraph_embedding'
cosine_similarities_dp = cosine_similarity(embeddings_dp, burnout_summary_embedding_dp)

#new df
df_dp = df.copy()

# Add the cosine similarities to the dataframe
df_dp['cosine_similarity'] = cosine_similarities_dp.flatten()

# Sort the dataframe by cosine similarity in descending order
df_dp_sorted = df_dp.sort_values('cosine_similarity', ascending=False)
# df_dp_sorted['recomm_domain'] = 'DP'
df_dp_sorted = df_dp_sorted[df_dp_sorted['Total_Views'] > 1000]

In [ ]:
df_dp_sorted.head(5)

,Search_Term,Video_URL,Video_Title,Channel_Name,Total_Subscribers,Total_Views,Total_Likes,Total_Dislikes,Total_Comments,Video_Description,...,Video_Tags,Age_Restrictions,License,Captions_Available,search_term_video_url,domain,Full_Des,video_length_min,preprocessed_text,cosine_similarity
290,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.898186
72,Building emotional resilience,https://www.youtube.com/watch?v=Lsd5oDUpDjc,Emotional Resiliency & Mental Toughness,SEALFIT,0,592942,9839,0,195,Website: https://www.sealfit.com\nFacebook: ht...,...,"['Masterty Program', 'SealFit Youtube Vignette...",NaN,youtube,NaN,Building emotional resilience_https://www.yout...,Emotional exhaustion,Website: https://www.sealfit.com\nFacebook: ht...,4.400000,website httpswwwsealfitcom facebook httpswwwfa...,0.890256
55,Coping with work pressure,https://www.youtube.com/watch?v=XxhuvNV-ync,Lecture 6.1: Stress and Coping,PSYC 4183 Health Psychology,unknown,34511,332,0,12,Table of Contents:\n\n01:51 - Emotion-focused ...,...,[],NaN,youtube,NaN,Coping with work pressure_https://www.youtube....,Emotional exhaustion,Table of Contents:\n\n01:51 - Emotion-focused ...,12.950000,table content 0151 emotionfocused coping 0309 ...,0.884296
294,Improving communication skills,https://www.youtube.com/watch?v=X3MG4fEJiF0,5 Simple Tips To Improve Your Communication Sk...,Ranveer Allahbadia,unknown,3151594,170268,0,3157,हमारी Personality Development Playlist देखो : ...,...,"['hindi', 'communication skills in hindi', 'be...",NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,हमारी Personality Development Playlist देखो : ...,8.716667,personality development playlist httpswwwyoutu...,0.883902
346,Work-life balance strategies,https://www.youtube.com/watch?v=hPz34pKUAhk,LOWEST Stress and BEST Work Life Balance. Cybe...,"Josh Madakor - Tech, Education, Career",unknown,52242,1805,0,251,🔒Hands-On Cybersecurity / SOC Analyst Training...,...,"['cyber security', 'cybersecurity', 'cyber sec...",NaN,youtube,NaN,Work-life balance strategies_https://www.youtu...,Personal achievement,🔒Hands-On Cybersecurity / SOC Analyst Training...,6.183333,handson cybersecurity soc analyst training rea...,0.871217


In [ ]:
# Initialize the BERT model for sentence embeddings
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# Create sentence pairs for encoding
sentence_pairs_pa = [(text, concise_paragraph_pa) for text in df['Full_Des']]

# Encode the sentence pairs
embeddings_pa = model.encode(sentence_pairs_pa)

In [ ]:
# Extract the embedding for the burnout_summary
burnout_summary_embedding_pa = model.encode([(concise_paragraph_pa, concise_paragraph_pa)])[0]

# Reshape the 'burnout_summary_embedding' to match the shape of 'embeddings'
burnout_summary_embedding_pa = burnout_summary_embedding_pa.reshape(1, -1)

# Calculate the cosine similarity between 'text' embeddings and 'concise_paragraph_embedding'
cosine_similarities_pa = cosine_similarity(embeddings_pa, burnout_summary_embedding_pa)

#new df
df_pa = df.copy()

# Add the cosine similarities to the dataframe
df_pa['cosine_similarity'] = cosine_similarities_pa.flatten()

# Sort the dataframe by cosine similarity in descending order
df_pa_sorted = df_pa.sort_values('cosine_similarity', ascending=False)
# df_pa_sorted['recomm_domain'] = 'PA'
df_pa_sorted = df_pa_sorted[df_pa_sorted['Total_Views'] > 1000]

In [ ]:
df_pa_sorted.head(5)

,Search_Term,Video_URL,Video_Title,Channel_Name,Total_Subscribers,Total_Views,Total_Likes,Total_Dislikes,Total_Comments,Video_Description,...,Video_Tags,Age_Restrictions,License,Captions_Available,search_term_video_url,domain,Full_Des,video_length_min,preprocessed_text,cosine_similarity
290,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.886499
346,Work-life balance strategies,https://www.youtube.com/watch?v=hPz34pKUAhk,LOWEST Stress and BEST Work Life Balance. Cybe...,"Josh Madakor - Tech, Education, Career",unknown,52242,1805,0,251,🔒Hands-On Cybersecurity / SOC Analyst Training...,...,"['cyber security', 'cybersecurity', 'cyber sec...",NaN,youtube,NaN,Work-life balance strategies_https://www.youtu...,Personal achievement,🔒Hands-On Cybersecurity / SOC Analyst Training...,6.183333,handson cybersecurity soc analyst training rea...,0.868956
294,Improving communication skills,https://www.youtube.com/watch?v=X3MG4fEJiF0,5 Simple Tips To Improve Your Communication Sk...,Ranveer Allahbadia,unknown,3151594,170268,0,3157,हमारी Personality Development Playlist देखो : ...,...,"['hindi', 'communication skills in hindi', 'be...",NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,हमारी Personality Development Playlist देखो : ...,8.716667,personality development playlist httpswwwyoutu...,0.868379
72,Building emotional resilience,https://www.youtube.com/watch?v=Lsd5oDUpDjc,Emotional Resiliency & Mental Toughness,SEALFIT,0,592942,9839,0,195,Website: https://www.sealfit.com\nFacebook: ht...,...,"['Masterty Program', 'SealFit Youtube Vignette...",NaN,youtube,NaN,Building emotional resilience_https://www.yout...,Emotional exhaustion,Website: https://www.sealfit.com\nFacebook: ht...,4.400000,website httpswwwsealfitcom facebook httpswwwfa...,0.858459
303,Improving communication skills,https://www.youtube.com/watch?v=QGK7aTGHeIQ,How to improve Communication Skills | How to s...,Aman Dhattarwal,unknown,2277047,144068,0,3490,Promo Code - aman25 off\nURL - https://www.cam...,...,"['cbse', 'class 12', 'class 12 student', 'how ...",NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,Promo Code - aman25 off\nURL - https://www.cam...,13.300000,promo code aman25 url httpswwwcamblycomenstude...,0.856841


In [ ]:
# Get the top 10 values from each dataframe
top_10_df1 = df_ee_sorted.head(10)
top_10_df2 = df_dp_sorted.head(10)
top_10_df3 = df_pa_sorted.head(10)
new_df = pd.concat([top_10_df1, top_10_df2, top_10_df3], ignore_index=True)

In [ ]:
# new_df
# Function to select videos based on domain and level
def select_videos(domain_df, level):
    if level == "high":
        return domain_df.head(3)
    elif level == "moderate":
        return domain_df.head(2)
    elif level == "low":
        return domain_df.head(1)
    else:
        return pd.DataFrame()  # Empty dataframe if level is not recognized


In [ ]:
ee_level = "high"
dp_level = "high"
pa_level = "high"

selected_videos = []

# Select videos based on the levels for each domain
if ee_level == "high":
    selected_videos.append(select_videos(df_ee_sorted, ee_level))
if dp_level == "high":
    selected_videos.append(select_videos(df_dp_sorted, dp_level))
if pa_level == "high":
    selected_videos.append(select_videos(df_pa_sorted, pa_level))

if ee_level == "moderate":
    selected_videos.append(select_videos(df_ee_sorted, ee_level))
if dp_level == "moderate":
    selected_videos.append(select_videos(df_dp_sorted, dp_level))
if pa_level == "moderate":
    selected_videos.append(select_videos(df_pa_sorted, pa_level))

if ee_level == "low":
    selected_videos.append(select_videos(df_ee_sorted, ee_level))
if dp_level == "low":
    selected_videos.append(select_videos(df_dp_sorted, dp_level))
if pa_level == "low":
    selected_videos.append(select_videos(df_pa_sorted, pa_level))


In [ ]:
# Concatenate the selected videos from all domains
result = pd.concat(selected_videos)
result

,Search_Term,Video_URL,Video_Title,Channel_Name,Total_Subscribers,Total_Views,Total_Likes,Total_Dislikes,Total_Comments,Video_Description,...,Video_Tags,Age_Restrictions,License,Captions_Available,search_term_video_url,domain,Full_Des,video_length_min,preprocessed_text,cosine_similarity
72,Building emotional resilience,https://www.youtube.com/watch?v=Lsd5oDUpDjc,Emotional Resiliency & Mental Toughness,SEALFIT,0,592942,9839,0,195,Website: https://www.sealfit.com\nFacebook: ht...,...,"['Masterty Program', 'SealFit Youtube Vignette...",NaN,youtube,NaN,Building emotional resilience_https://www.yout...,Emotional exhaustion,Website: https://www.sealfit.com\nFacebook: ht...,4.400000,website httpswwwsealfitcom facebook httpswwwfa...,0.886579
290,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.883736
55,Coping with work pressure,https://www.youtube.com/watch?v=XxhuvNV-ync,Lecture 6.1: Stress and Coping,PSYC 4183 Health Psychology,unknown,34511,332,0,12,Table of Contents:\n\n01:51 - Emotion-focused ...,...,[],NaN,youtube,NaN,Coping with work pressure_https://www.youtube....,Emotional exhaustion,Table of Contents:\n\n01:51 - Emotion-focused ...,12.950000,table content 0151 emotionfocused coping 0309 ...,0.873610
290,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.898186
72,Building emotional resilience,https://www.youtube.com/watch?v=Lsd5oDUpDjc,Emotional Resiliency & Mental Toughness,SEALFIT,0,592942,9839,0,195,Website: https://www.sealfit.com\nFacebook: ht...,...,"['Masterty Program', 'SealFit Youtube Vignette...",NaN,youtube,NaN,Building emotional resilience_https://www.yout...,Emotional exhaustion,Website: https://www.sealfit.com\nFacebook: ht...,4.400000,website httpswwwsealfitcom facebook httpswwwfa...,0.890256
55,Coping with work pressure,https://www.youtube.com/watch?v=XxhuvNV-ync,Lecture 6.1: Stress and Coping,PSYC 4183 Health Psychology,unknown,34511,332,0,12,Table of Contents:\n\n01:51 - Emotion-focused ...,...,[],NaN,youtube,NaN,Coping with work pressure_https://www.youtube....,Emotional exhaustion,Table of Contents:\n\n01:51 - Emotion-focused ...,12.950000,table content 0151 emotionfocused coping 0309 ...,0.884296
290,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.886499
346,Work-life balance strategies,https://www.youtube.com/watch?v=hPz34pKUAhk,LOWEST Stress and BEST Work Life Balance. Cybe...,"Josh Madakor - Tech, Education, Career",unknown,52242,1805,0,251,🔒Hands-On Cybersecurity / SOC Analyst Training...,...,"['cyber security', 'cybersecurity', 'cyber sec...",NaN,youtube,NaN,Work-life balance strategies_https://www.youtu...,Personal achievement,🔒Hands-On Cybersecurity / SOC Analyst Training...,6.183333,handson cybersecurity soc analyst training rea...,0.868956
294,Improving communication skills,https://www.youtube.com/watch?v=X3MG4fEJiF0,5 Simple Tips To Improve Your Communication Sk...,Ranveer Allahbadia,unknown,3151594,170268,0,3157,हमारी P

In [ ]:
# # Sort the dataframe by the similarity score column in descending order
# sorted_df = new_df.sort_values('cosine_similarity', ascending=False)

# Retrieve the top 9 unique records based on the similarity score
top_9_unique_records = result.drop_duplicates(subset='Video_URL').head(9)
# top_9_unique_records = top_9_unique_records.sort_values('total_views', ascending=False)

# Reset the index of the new dataframe
top_9_unique_records.reset_index(drop=True, inplace=True)

In [ ]:
top_9_unique_records

,Search_Term,Video_URL,Video_Title,Channel_Name,Total_Subscribers,Total_Views,Total_Likes,Total_Dislikes,Total_Comments,Video_Description,...,Video_Tags,Age_Restrictions,License,Captions_Available,search_term_video_url,domain,Full_Des,video_length_min,preprocessed_text,cosine_similarity
0,Building emotional resilience,https://www.youtube.com/watch?v=Lsd5oDUpDjc,Emotional Resiliency & Mental Toughness,SEALFIT,0,592942,9839,0,195,Website: https://www.sealfit.com\nFacebook: ht...,...,"['Masterty Program', 'SealFit Youtube Vignette...",NaN,youtube,NaN,Building emotional resilience_https://www.yout...,Emotional exhaustion,Website: https://www.sealfit.com\nFacebook: ht...,4.400000,website httpswwwsealfitcom facebook httpswwwfa...,0.886579
1,Improving communication skills,https://www.youtube.com/watch?v=HCQqeKgIa_Q,Improve Your Speaking & Communication Skills a...,Mr Forged,unknown,183494,11438,0,111,#masculinity #selfimprovement #speaking \n\nTi...,...,[],NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,#masculinity #selfimprovement #speaking \n\nTi...,8.266667,masculinity selfimprovement speaking time code...,0.883736
2,Coping with work pressure,https://www.youtube.com/watch?v=XxhuvNV-ync,Lecture 6.1: Stress and Coping,PSYC 4183 Health Psychology,unknown,34511,332,0,12,Table of Contents:\n\n01:51 - Emotion-focused ...,...,[],NaN,youtube,NaN,Coping with work pressure_https://www.youtube....,Emotional exhaustion,Table of Contents:\n\n01:51 - Emotion-focused ...,12.950000,table content 0151 emotionfocused coping 0309 ...,0.873610
3,Work-life balance strategies,https://www.youtube.com/watch?v=hPz34pKUAhk,LOWEST Stress and BEST Work Life Balance. Cybe...,"Josh Madakor - Tech, Education, Career",unknown,52242,1805,0,251,🔒Hands-On Cybersecurity / SOC Analyst Training...,...,"['cyber security', 'cybersecurity', 'cyber sec...",NaN,youtube,NaN,Work-life balance strategies_https://www.youtu...,Personal achievement,🔒Hands-On Cybersecurity / SOC Analyst Training...,6.183333,handson cybersecurity soc analyst training rea...,0.868956
4,Improving communication skills,https://www.youtube.com/watch?v=X3MG4fEJiF0,5 Simple Tips To Improve Your Communication Sk...,Ranveer Allahbadia,unknown,3151594,170268,0,3157,हमारी Personality Development Playlist देखो : ...,...,"['hindi', 'communication skills in hindi', 'be...",NaN,youtube,NaN,Improving communication skills_https://www.you...,Depersonalization,हमारी Personality Development Playlist देखो : ...,8.716667,personality development playlist httpswwwyoutu...,0.868379


In [ ]:
 top_9_unique_records.to_csv('HHH_M_CR_Akash_OldD.csv', index=False)

In [ ]:
# num_videos = 15

# # Select videos based on the levels for each domain
# ee_videos = select_videos(df_ee.sample(frac=1).reset_index(drop=True).iloc[:num_videos], ee_level)
# dp_videos = select_videos(df_dp.sample(frac=1).reset_index(drop=True).iloc[:num_videos], dp_level)
# pa_videos = select_videos(df_pa.sample(frac=1).reset_index(drop=True).iloc[:num_videos], pa_level)

# # Concatenate the selected videos from all domains
# results = pd.concat([ee_videos, dp_videos, pa_videos])

# # Remove duplicates
# resultss = results.drop_duplicates().reset_index(drop=True)

# # print(result)
# resultss.head()

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
# Load the dataset
# df = pd.read_csv('your_dataset.csv')

# Preprocess the text
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove HTML tags
    text = BeautifulSoup(text, 'lxml').get_text()
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if not w in stop_words]
    text = ' '.join(filtered_text)
    return text

df['Video_Title'] = df['Video_Title'].apply(preprocess_text)
df['Video_Description'] = df['Video_Description'].apply(preprocess_text)

# Load Universal Sentence Encoder (USE) module
use_module = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Define a function to calculate similarity scores using USE embeddings
def calculate_similarity_use(query, corpus):
    query_embedding = use_module([query])[0].numpy()
    corpus_embeddings = use_module(corpus).numpy()
    similarity_scores = cosine_similarity([query_embedding], corpus_embeddings)[0]
    return similarity_scores

def calculate_diversity_score(similarity_scores):
    diversity_score = 1 - np.std(similarity_scores)
    return diversity_score

queries = [
    "Vikrant is a mid-aged male sales executive who is facing a moderate amount of emotional exhaustion. While his job requires a lot of effort and direct contact with people, his level of emotional exhaustion is still quite manageable. He feels emotionally drained a few times per month, and frustrated with his work a few times per year. He also feels like he works too hard and is stressed by working with people once a week. Despite the fact that he is facing emotional exhaustion, he is still able to manage it with a few strategies. He should focus on taking regular breaks from work and engaging in activities that help him relax and unwind. Additionally, he should also try to find ways to meaningfully connect with people in a way that doesn't feel draining. By combining rest, relaxation, and meaningful social connections, Vikrant can better manage his emotional exhaustion.",
    "Vikrant is a mid-age male Sales Executive who, on the whole, exhibits a healthy relationship with his co-workers and team. He has answered all questions pertaining to depersonalization in a manner that indicates he is fairly connected with his team and colleagues and has a good level of empathy towards them. He has only been feeling tired a few times a month, and has had the impression that his team has made him responsible for their issues at a few times per month. His patience has only been at the end of its rope a few times per year, and he has not become more uncaring, insensitive, or afraid that his job is making him uncaring. To ensure Vikrant continues to have a healthy work relationship, he should make sure that he takes regular breaks during the day, takes the time to connect with his team/colleagues on a personal level, and makes sure to prioritize his own mental health and well-being.",
    "Vikrant is a mid-aged male Sales Executive who is highly successful in his role. He is able to deal with the emotional and practical issues that come with the job in a calm and effective manner. He has strong communication skills and is able to easily understand the feelings of his team and colleagues. He often looks after their problems in an efficient and diplomatic manner. He also takes time to create a relaxed atmosphere with his team and colleagues. Once a week, he takes time to accomplish worthwhile tasks, feel full of energy, and feel refreshed when close to his team/colleagues. He also makes sure to take a few times a month to reflect on how his work positively influences people. His strategy to success includes communication, creating a relaxed atmosphere, taking the time to help his team and colleagues, and reflecting on how his work is contributing positively in the world."
]

df_copy = df.copy()

# Calculate similarity scores for each paragraph
for query in queries:
    df_copy['Similarity_Score'] = calculate_similarity_use(query, df_copy['Video_Title'] + ' ' + df_copy['Video_Description'])
    # Sort the dataframe by similarity score in descending order
    df_sorted = df_copy.sort_values(by='Similarity_Score', ascending=False)
    # Get recommended video URLs and similarity scores for top 15 videos
    recommended_urls = df_sorted['Video_URL'].head(15).tolist()
    similarity_scores = df_sorted['Similarity_Score'].head(15).tolist()
    diversity_score = calculate_diversity_score(similarity_scores)
    print("Query: {}".format(query))
    print("Diversity Score: {}".format(diversity_score))
    print("Top 15 Recommended Videos:")
    for url, score in zip(recommended_urls, similarity_scores):
        print("URL: {}, Similarity Score: {}".format(url, score))
    print("-----------------------")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Query: Vikrant is a mid-aged male sales executive who is facing a moderate amount of emotional exhaustion. While his job requires a lot of effort and direct contact with people, his level of emotional exhaustion is still quite manageable. He feels emotionally drained a few times per month, and frustrated with his work a few times per year. He also feels like he works too hard and is stressed by working with people once a week. Despite the fact that he is facing emotional exhaustion, he is still able to manage it with a few strategies. He should focus on taking regular breaks from work and engaging in activities that help him relax and unwind. Additionally, he should also try to find ways to meaningfully connect with people in a way that doesn't feel draining. By combining rest, relaxation, and meaningful social connections, Vikrant can better manage his emotional exhaustion.
Diversity Score: 0.9766468263537528
Top 15 Recommended Videos:
URL: https://www.youtube.com/watch?v=UE_H9QhGktE,